In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader.data as web
import timeit

pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.4f}'.format

In [2]:
helper_dir = '../helper'

%run {helper_dir}/utils.py
%run {helper_dir}/rolling_funcs.py
%run {helper_dir}/rolling_apply_funcs.py

In [3]:
from statsmodels.tsa.stattools import coint

def coint_pval(s1: Union[np.ndarray, np.array, pd.Series], s2: Union[np.ndarray, np.array, pd.Series]): 
    _, pval, _ = coint(s1, s2)
    return pval

## Download data

In [4]:
# download stock prices
start = dt.date(2017, 1, 1)
end = dt.date(2022, 1, 1)
tickers = ['ABG', 'ASTI', 'DQ', 'FSLR', 'SPY']
daily_rets = web.get_data_yahoo(tickers, start, end)['Adj Close'].pct_change().dropna()
daily_rets.head()

Symbols,ABG,ASTI,DQ,FSLR,SPY
Date,,,,,
2017-01-04,0.0240,-0.1515,-0.0045,0.0188,0.0059
2017-01-05,-0.0188,-0.1107,0.0233,-0.0085,-0.0008
2017-01-06,-0.0167,-0.1566,0.0124,-0.0165,0.0036
2017-01-09,0.0081,-0.1476,0.0068,0.0075,-0.0033
2017-01-10,0.0193,-0.0503,0.0005,-0.0054,0.0000


In [5]:
daily_rets.tail()

Symbols,ABG,ASTI,DQ,FSLR,SPY
Date,,,,,
2021-12-27,0.0187,-0.1724,-0.0344,0.0056,0.0142
2021-12-28,0.0121,0.3333,0.0059,-0.0058,-0.0008
2021-12-29,-0.0031,-0.0312,0.0016,-0.0019,0.0013
2021-12-30,0.0064,-0.1290,0.0811,0.0099,-0.0028
2021-12-31,0.0155,-0.0741,-0.0151,-0.0100,-0.0025


## Input

In [6]:
ndays = 30
var1 = 'DQ'
var2 = 'FSLR'

## Calculate p-value of Cointegration test

In [7]:
pvals_01 = roll(daily_rets, ndays).apply(lambda x: coint_pval(x[var1], x[var2]))
pvals_02 = pd.concat([pd.Series(coint_pval(subdf[var1], subdf[var2]), index=[subdf.index[-1]]) for subdf in groll(daily_rets, ndays)])
pvals_03 = rolling_apply_pd(daily_rets[var1], daily_rets[var2], ndays, coint_pval)
pvals_04 = rolling_apply_np(daily_rets[var1], daily_rets[var2], ndays, coint_pval)

In [8]:
print(pvals_01.head(), '\n\n')
print(pvals_02.head(), '\n\n')
print(pvals_03.head(), '\n\n')
print(pvals_04.dropna().head(), '\n\n')

Date
2017-02-15   0.0000
2017-02-16   0.0000
2017-02-17   0.0000
2017-02-21   0.0000
2017-02-22   0.0000
dtype: float64 


2017-02-15   0.0000
2017-02-16   0.0000
2017-02-17   0.0000
2017-02-21   0.0000
2017-02-22   0.0000
dtype: float64 


Date
2017-02-15   0.0000
2017-02-16   0.0000
2017-02-17   0.0000
2017-02-21   0.0000
2017-02-22   0.0000
dtype: float64 


Date
2017-02-15   0.0000
2017-02-16   0.0000
2017-02-17   0.0000
2017-02-21   0.0000
2017-02-22   0.0000
dtype: float64 




In [9]:
print(pvals_01.tail(), '\n\n')
print(pvals_02.tail(), '\n\n')
print(pvals_03.tail(), '\n\n')
print(pvals_04.tail(), '\n\n')

Date
2021-12-27   0.0038
2021-12-28   0.0008
2021-12-29   0.0009
2021-12-30   0.0168
2021-12-31   0.0181
dtype: float64 


2021-12-27   0.0038
2021-12-28   0.0008
2021-12-29   0.0009
2021-12-30   0.0168
2021-12-31   0.0181
dtype: float64 


Date
2021-12-27   0.0038
2021-12-28   0.0008
2021-12-29   0.0009
2021-12-30   0.0168
2021-12-31   0.0181
dtype: float64 


Date
2021-12-27   0.0038
2021-12-28   0.0008
2021-12-29   0.0009
2021-12-30   0.0168
2021-12-31   0.0181
dtype: float64 




In [10]:
print(equal(pvals_01, pvals_02))
print(equal(pvals_02, pvals_03))
print(equal(pvals_02, pvals_04))

True
True
True


In [11]:
%%timeit
pvals_01 = roll(daily_rets, ndays).apply(lambda x: coint_pval(x[var1], x[var2]))

6.94 s ± 6.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
pvals_02 = pd.concat([pd.Series(coint_pval(subdf[var1], subdf[var2]), index=[subdf.index[-1]]) for subdf in groll(daily_rets, ndays)])

7.4 s ± 51.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
pvals_03 = rolling_apply_pd(daily_rets[var1], daily_rets[var2], ndays, coint_pval)

7.06 s ± 60 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
pvals_04 = rolling_apply_np(daily_rets[var1], daily_rets[var2], ndays, coint_pval)

6.77 s ± 42.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### All methods are pretty slow, and `rolling_apply_np()` is supposed to be fast. The bottleneck: statsmodels `coint()` is slow. To speed up, we'd need to write a faster version of `coint()`.